In [1]:
import pandas as pd

In [2]:
filename = 'app_reviews_airvisual-air-quality-forecast_1048912974.json'

In [4]:
df = pd.read_json(filename)

In [5]:
df.head()

,review_id,rating,title,review_date,user_name,review,response_id,dev_response,response_date
0,1512229481,5,Parfait,2016-12-30T22:00:51Z,PaysDuMontBlancPollué,"Appli indispensable au pays du Mont-Blanc, et ...",NaN,NaN,NaN
1,4004281790,5,Très utile !,2019-04-12T20:00:15Z,lucasfinck1211,Cela fait désormais une semaine que j’ai téléc...,NaN,NaN,NaN
2,1513779072,5,Pays de Savoie -> utile!!!,2017-01-02T09:00:04Z,Pornawak333,"Habitant d Annecy, ville magnifique mais forte...",NaN,NaN,NaN
3,1517811476,5,Indispensable,2017-01-08T11:50:03Z,pascalwirth,"Très bonne application, une démarche réseau af...",NaN,NaN,NaN
4,1508935600,4,Recherche de ville,2016-12-25T20:42:43Z,YodaMoi,Très bonne appli. mais la recherche de ville m...,NaN,NaN,NaN


In [6]:
df.columns

Index(['review_id', 'rating', 'title', 'review_date', 'user_name', 'review',
       'response_id', 'dev_response', 'response_date'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   review_id      334 non-null    int64  
 1   rating         334 non-null    int64  
 2   title          334 non-null    object 
 3   review_date    334 non-null    object 
 4   user_name      334 non-null    object 
 5   review         334 non-null    object 
 6   response_id    3 non-null      float64
 7   dev_response   3 non-null      object 
 8   response_date  3 non-null      object 
dtypes: float64(1), int64(2), object(6)
memory usage: 23.6+ KB


In [8]:
df['review']

0      Appli indispensable au pays du Mont-Blanc, et ...
1      Cela fait désormais une semaine que j’ai téléc...
2      Habitant d Annecy, ville magnifique mais forte...
3      Très bonne application, une démarche réseau af...
4      Très bonne appli. mais la recherche de ville m...
                             ...                        
329                                              Parfait
330                                              parfait
331    You should publish on Facebook . The governmen...
332                                                 Bien
333               Very usefull and seems to be accurate.
Name: review, Length: 334, dtype: object

In [13]:
df.sort_values(by='review_date', inplace = True, ascending = False)

In [19]:
df.head(20)

,review_id,rating,title,review_date,user_name,review,response_id,dev_response,response_date
299,6104087393,5,De l’air,2020-06-21T11:24:10Z,Terre de sienne,Tip top,NaN,NaN,NaN
133,6085970206,5,La Maison Blanche,2020-06-17T06:10:30Z,tsm energies services,L’appareil est très bien fait,NaN,NaN,NaN
253,6045921703,5,Top 👍,2020-06-07T08:24:01Z,Sécu2007,"Très bonne application, très utile pour connaî...",NaN,NaN,NaN
302,5977414552,4,Very useful,2020-05-22T04:02:32Z,Mattiouz,Very useful but improvements would be required...,NaN,NaN,NaN
233,5974704788,5,Très bien,2020-05-21T13:31:13Z,Cjchavanne,Très bien,NaN,NaN,NaN
192,5935245643,5,Pratique,2020-05-12T10:32:33Z,ManuLes,Application très pratique pour savoir si l’air...,NaN,NaN,NaN
82,5928224600,5,Indispensable pour comprendre la planète !,2020-05-10T16:15:52Z,Zlyzinc,Application indispensable à donner - notamment...,NaN,NaN,NaN
284,5896292691,5,One of the best coverage,2020-05-03T06:33:25Z,georgium,I live in Asia it is the only App which shows ...,NaN,NaN,NaN
206,5890657131,4,Jam470,2020-05-01T23:39:27Z,jamo47000,Bonne application bien dans ensemble juste un ...,NaN,NaN,NaN
101,5880383779,5,Pure,2020-04-29T16:28:33Z,LeRobuste,Utile pour choisir son lieu d’habitation !,NaN,NaN,NaN


In [18]:
df.to_csv('app_reviews_airvisual-air-quality-forecast_1048912974.csv', encoding='utf-8-sig', sep =';')